# Grewpy tutorial: Modify data

https://grew.fr/grewpy/modify_data/

In [ ]:
import grewpy
from grewpy import Corpus, CorpusDraft, Request

grewpy.set_config("ud")  # ud or basic
corpus = Corpus("../UD_output.conllu")

## Access data in a corpus

In [ ]:
# Access to the corpus
sentence = corpus[1]
print("A corpus is a set of graphs:", type(sentence))

In [ ]:
# Each graph is a sentence and contains all its information
print("Sentence metadata:")
sentence.meta

In [ ]:
# Sentence order, which in this case is the same as the token's id
print(sentence.order)

In [ ]:
# Token features, which make possible to access every token feature
print(sentence.features)

# e.g get all upos of the sentence
print([sentence.features[id]["upos"] for id in sentence.features if id != "0"])

In [ ]:
# It's possible to access to edges between nodes as successors
print(sentence.sucs)

## Modifying a corpus
`Corpus` is an abstract object which cannot be modified directly:

In [ ]:
try:
    corpus[0] = corpus[1]
except TypeError as error_message:
    print(f"{error_message}")

`CorpusDraft` is an object similar to `Corpus` but which is mutable.
Below, we add the feature `Transitive=Yes` to all occurrences of verbs with a direct object.

1. We make the search on `corpus` (an instance of `Corpus`).
2. The modification is done on a `CorpusDraft` counterpart named `draft`.
3. The `draft` should be transformed again into a `Corpus` (names `corpus2` below) in order to use the `count` method.

In [ ]:
# step 1
req7 = Request().pattern("X[upos=VERB]; Y[upos=NOUN|PROPN|PRON]; X-[comp:obj]->Y")
occurrences = corpus.search(req7)

# step 2
draft = CorpusDraft(corpus)
for occ in occurrences:
    sent_id = occ["sent_id"]
    verb_node_id = occ["matching"]["nodes"]["X"]
    draft[sent_id][verb_node_id].update({"Transitive": "Yes"})

# step 3
corpus2 = Corpus(draft)
corpus2.count(Request("pattern { X[Transitive=Yes] }"))

It's possible to modify a whole `CorpusDraft` with a function getting a graph as input.

In [ ]:
def relabel_noun(graph):
    for node in graph:
        if "upos" in graph[node] and graph[node]["upos"] == "NOUN":
            graph[node]["upos"] = "N"
    return graph


draft3 = draft.map(relabel_noun)
# Note that the map function has replaced the apply function which is deprecated in 0.6


# Again, we need to turn the result into a `Corpus` before using the `count` method.
corpus3 = Corpus(draft3)
corpus3.count(Request("pattern { X[upos=N] }"))

## Modifying a corpus using a GRS (Graph Rewriting System)
In many cases, it is not required to uses a `CorpusDraft` and the modification of a corpus can be encoded with graph rewriting rules.

The example above (identifying transitive verbs) can be rephrased as below.
See TODO link for an explanation of the `without` clause in this example.

In [ ]:
from grewpy import GRS

s = """
strat main { Onf(tv) }

rule tv {
  pattern { X[upos=VERB]; Y[upos=NOUN|PROPN|PRON]; X-[comp:obj]->Y }
  without { X[Transitive = Yes] }
  commands { X.Transitive = Yes }
}
"""
grs = GRS(s)
corpus2bis = grs.apply(corpus)
corpus2bis.count(Request("pattern { X[Transitive=Yes] }"))

For the example, where the upos tag `NOUN` is changed to `N`, this can be done with a GRS:

In [ ]:
from grewpy import GRS

grs3 = GRS("""
strat main { Onf(noun2n) }

rule noun2n {
  pattern { X[upos=NOUN] }
  commands { X.upos = N }
}
""")
corpus3bis = grs3.apply(corpus)
corpus3bis.count(Request("pattern { X[upos=N] }"))

Similarily to the `CorpusDraft` above, there is a module `GRSDraft` which can be inspected and which is mutable.

In [ ]:
from grewpy import GRSDraft

s = """
strat main {Onf(cxns)}
package cxns {
    rule existential {
        pattern {X-[comp@expl]->Y; X[lemma=be]}
        without {X[Cxn=Existential]}
        commands {X.Cxn=Existential}
    }
}
"""

grs_draft = GRSDraft(s)

for rule in grs_draft["cxns"].rules():
    print(f"{rule=}")

A `GRSDraft` cannot be applied to a corpus, it should be turned into a `GRS`:

In [ ]:
grs = GRS(grs_draft)
corpus.apply(grs)
n_existentials = corpus.count(Request("pattern { X[Cxn=Existential] }"))
print(f"{n_existentials=}")